<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-09 09:49:17
-------------------
qualified stocks: 88
with latest results: 29
still star stocks: 16
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.11 L
Current:  1.35 C
-------------------
Today PnL: 1.14 L (0.85%)
Current PnL: -15.21 L (-10.6%)
CY Booked + Current PnL: -9.12 L (-6.36%)
-------------------
Total profit:  3.78 L
Total loss:  -18.99 L
-------------------
Total Booked + Current PnL: 17.86 L (15.32%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.52%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 70.14 L (52.15%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LICI,785.05,983.0,-2.96,H-LC,85.23,238464.0,43772.0,5318.0,0.18,22.48,2.23,25.21,31.0,8.23,1.90,31.86,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.39,104278.0,25916.0,8113.0,-0.20,33.07,7.78,43.43,71.0,3.19,0.83,39.47,XY24,NTT,MISC
57,RELIANCE,1291.83,1526.0,6.50,X-LC,36.36,157996.0,17187.0,8342.0,0.42,12.21,5.28,18.13,20.0,2.06,1.26,24.75,XY25,NTT,REFINERIES
80,UNIONBANK,123.87,163.0,-0.31,M-LC,93.18,174279.0,33439.0,11049.0,0.48,23.74,6.34,31.59,81.0,3.03,1.39,51.39,XY24,NTT,BANKS
73,TATAELXSI,7332.28,7332.0,-12.79,X-MC,64.77,91595.0,-11057.0,11056.0,1.07,-10.77,12.07,-0.00,51.0,-1.00,0.73,37.68,OX40N,NTT,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,53.41,62938.0,-50611.0,90662.0,9.01,-44.57,144.05,35.27,246.0,-0.56,0.50,54.04,XR,NTT,HOTELS
58,REPCOHOME,515.07,880.00,-52.31,H-SC,80.68,176274.0,-28724.0,173965.0,5.31,-14.01,98.69,70.85,123.0,-0.17,1.41,42.30,XY24,NTT,FINANCE
55,RBLBANK,230.61,388.00,49.48,M-SC,75.00,231912.0,-2388.0,162292.0,4.88,-1.02,69.98,68.25,229.0,-0.01,1.85,52.72,XY24,NTT,BANKS
29,HAPPSTMNDS,919.80,1491.84,-23.30,M-SC,42.05,64601.0,-31978.0,92044.0,4.47,-33.11,142.48,62.19,195.0,-0.35,0.52,13.27,AR,ATH,IT
11,BANDHANBNK,214.25,400.00,-29.75,H-SC,73.86,209658.0,-38872.0,254336.0,4.14,-15.64,121.31,86.70,118.0,-0.15,1.67,38.70,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,558.54,680.00,-15.35,H-MC,43.18,181546.0,4489.0,34022.0,-1.51,2.54,18.74,21.75,106.0,0.13,1.45,29.86,XY24,NTT,PAINTS
37,INDIAMART,2327.09,4871.06,-51.09,H-SC,46.59,127534.0,4198.0,130633.0,-0.90,3.40,102.43,109.32,111.0,0.03,1.02,25.99,AR,ATH,MISC
17,CAMPUS,294.86,393.00,NaN,NaN,87.50,168623.0,848.0,54988.0,-0.84,0.51,32.61,33.28,188.0,0.02,1.34,30.55,XY24,NTT,FOOTWEAR
67,SIS,477.00,477.00,2203.81,M-SC,60.23,62365.0,-18725.0,18728.0,-0.84,-23.09,30.03,0.00,211.0,-1.00,0.50,26.39,OX40N,NTT,MISC
6,APOLLOHOSP,6904.43,8285.00,-16.22,H-LC,22.73,103522.0,-44.0,20756.0,-0.59,-0.04,20.05,20.00,33.0,-0.00,0.83,14.28,X40N,BTT,HEALTHCARE


In [12]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,52.77,53.0,49.36,M-SC,77.27,129222.0,-13257.0,13878.0,0.06,-9.30,10.74,0.44,227.0,-0.96,1.03,52.52,OX40N,NTT,BANKS
73,TATAELXSI,7332.28,7332.0,-12.79,X-MC,64.77,91595.0,-11057.0,11056.0,1.07,-10.77,12.07,-0.00,51.0,-1.00,0.73,37.68,OX40N,NTT,IT
71,SYMPHONY,1306.42,1306.0,-28.42,M-SC,1.14,152405.0,-18736.0,18685.0,1.11,-10.95,12.26,-0.03,178.0,-1.00,1.22,9.54,OX40N,NTT,DURABLES
45,KANSAINER,299.63,340.0,-67.85,H-SC,7.95,224100.0,-45567.0,81909.0,-0.06,-16.90,36.55,13.47,134.0,-0.56,1.79,12.17,XY24,NTT,PAINTS
78,TTKPRESTIG,769.29,770.0,-10.66,X-SC,17.05,81718.0,-19059.0,19155.0,0.39,-18.91,23.44,0.09,64.0,-0.99,0.65,6.34,OX40N,NTT,DURABLES


In [13]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,3794.03,4998.00,-18.09,X-LC,13.64,266651.0,-29283.0,123193.0,1.02,-9.90,46.20,31.73,1.0,-0.24,2.13,6.73,X40,BTT,IT
40,INFY,1461.46,2275.00,-11.70,X-LC,40.91,283285.0,21684.0,123937.0,1.20,8.29,43.75,55.67,2.0,0.17,2.26,14.90,X40,BTT,IT
49,LTIM,5564.16,7197.33,-1.63,X-LC,65.91,189144.0,-11166.0,69964.0,0.96,-5.57,36.99,29.35,3.0,-0.16,1.51,30.67,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.52,X-LC,9.09,152177.0,-49209.0,136822.0,-0.33,-24.44,89.91,43.51,5.0,-0.36,1.21,4.96,X40,BTT,PAINTS
33,HINDUNILVR,2413.81,2723.00,-15.01,X-LC,20.45,266795.0,-3552.0,38178.0,-0.32,-1.31,14.31,12.81,6.0,-0.09,2.13,10.42,XY25,NTT,FMCG


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UNIONBANK,123.87,163.0,-0.31,M-LC,93.18,174279.0,33439.0,11049.0,0.48,23.74,6.34,31.59,81.0,3.03,1.39,51.39,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.39,104278.0,25916.0,8113.0,-0.20,33.07,7.78,43.43,71.0,3.19,0.83,39.47,XY24,NTT,MISC
48,LICI,785.05,983.0,-2.96,H-LC,85.23,238464.0,43772.0,5318.0,0.18,22.48,2.23,25.21,31.0,8.23,1.90,31.86,XY25,NTT,INSURANCE


In [15]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,130.64,228.00,105.11,M-MC,89.77,233607.0,8645.0,159016.0,0.46,3.84,68.07,74.53,176.0,0.05,1.86,35.63,XY24,BTT,STEEL
18,CAMS,3643.00,5250.99,7.33,H-SC,88.64,120638.0,18634.0,26396.0,1.09,18.27,21.88,44.14,115.0,0.71,0.96,37.48,X40N,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,3.96,M-LC,57.39,104278.0,25916.0,8113.0,-0.20,33.07,7.78,43.43,71.0,3.19,0.83,39.47,XY24,NTT,MISC
80,UNIONBANK,123.87,163.00,-0.31,M-LC,93.18,174279.0,33439.0,11049.0,0.48,23.74,6.34,31.59,81.0,3.03,1.39,51.39,XY24,NTT,BANKS
87,WIPRO,243.46,420.00,-9.26,M-LC,28.41,155558.0,4613.0,104846.0,0.93,3.06,67.40,72.51,68.0,0.04,1.24,7.68,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,113.49,190.00,-28.22,M-MC,76.14,141544.0,12392.0,74679.0,0.10,9.60,52.76,67.42,158.0,0.17,1.13,37.66,XR,NTT,BANKS
86,WHIRLPOOL,1167.49,2270.00,-33.53,M-SC,47.73,193820.0,22199.0,139880.0,2.34,12.93,72.17,94.43,183.0,0.16,1.55,43.05,XR,NTT,DURABLES
87,WIPRO,243.46,420.00,-9.26,M-LC,28.41,155558.0,4613.0,104846.0,0.93,3.06,67.40,72.51,68.0,0.04,1.24,7.68,XR,NTT,IT
37,INDIAMART,2327.09,4871.06,-51.09,H-SC,46.59,127534.0,4198.0,130633.0,-0.90,3.40,102.43,109.32,111.0,0.03,1.02,25.99,AR,ATH,MISC
1,AARTIIND,487.04,919.00,39.94,M-SC,39.77,80285.0,-1051.0,73188.0,0.82,-1.29,91.16,88.69,213.0,-0.01,0.64,35.38,XR,NTT,CHEMICALS


In [17]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.0,-57.75,L-SC,53.41,62938.0,-50611.0,90662.0,9.01,-44.57,144.05,35.27,246.0,-0.56,0.50,54.04,XR,NTT,HOTELS
50,MASFIN,326.60,399.5,-20.96,H-SC,69.32,89760.0,-8220.0,30088.0,1.51,-8.39,33.52,22.32,127.0,-0.27,0.72,30.18,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.39,104278.0,25916.0,8113.0,-0.20,33.07,7.78,43.43,71.0,3.19,0.83,39.47,XY24,NTT,MISC
82,VALIANTORG,512.64,838.0,-367.62,H-SC,100.00,113867.0,-19419.0,104018.0,1.45,-14.57,91.35,63.47,135.0,-0.19,0.91,90.87,XR,NTT,CHEMICALS
70,SURYODAY,189.78,240.0,66.67,H-SC,94.32,117750.0,-32176.0,71851.0,2.83,-21.46,61.02,26.46,132.0,-0.45,0.94,50.85,XR,NTT,BANKS


In [18]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,512.64,838.00,-367.62,H-SC,100.00,113867.0,-19419.0,104018.0,1.45,-14.57,91.35,63.47,135.0,-0.19,0.91,90.87,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.00,6638.89,M-SC,98.86,73931.0,-19879.0,96495.0,0.71,-21.19,130.52,81.67,222.0,-0.21,0.59,43.17,XR,NTT,CERAMICS
76,TITAGARH,1097.23,1548.00,-6.04,H-SC,97.73,164938.0,-27077.0,105956.0,0.17,-14.10,64.24,41.08,131.0,-0.26,1.32,39.60,XY24,NTT,ENGINEERING
72,TANLA,917.30,1963.11,-32.99,H-SC,96.59,164234.0,-75181.0,348143.0,1.03,-31.40,211.98,114.01,117.0,-0.22,1.31,50.63,AR,ATH,IT
69,STARHEALTH,551.56,761.00,3.93,M-SC,95.45,170011.0,-30205.0,106240.0,-0.35,-15.09,62.49,37.97,202.0,-0.28,1.36,36.51,XY24,NTT,INSURANCE


In [19]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.44
1,25,43.81
2,50,73.37


In [20]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     33.65
XY25     15.40
XR       11.55
X40      10.64
X40N      8.93
OX40N     8.76
AR        8.36
X200      1.51
SR        1.24
Name: CurrAlloc%, dtype: float64

In [21]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.64
X-LC    19.03
M-SC    17.49
H-MC     8.22
M-LC     8.16
H-LC     7.47
M-MC     5.80
X-MC     5.78
X-SC     2.91
L-SC     2.47
L-LC     1.18
Name: CurrAlloc%, dtype: float64

In [22]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.10,-1.39,33.14
IT,10.83,-14.93,75.63
BANKS,10.50,-6.39,53.87
MISC,6.82,-26.48,75.94
FINANCE,6.40,-19.25,65.76
INSURANCE,5.58,4.69,23.79
PAINTS,5.52,-18.92,42.41
ELECTRICAL,5.42,-3.41,48.26
HEALTHCARE,4.57,-3.92,34.93


In [23]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2754544.0,26
XR,1189976.0,14
AR,1024352.0,9
X40,595138.0,7
XY25,448026.0,10
X40N,409203.0,8
OX40N,333336.0,11
SR,189403.0,2
X200,69964.0,1


In [24]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1906872.0,18
M-SC,1685624.0,20
X-LC,827465.0,11
M-MC,541115.0,5
H-MC,467706.0,7
L-SC,286513.0,4
X-MC,284710.0,5
M-LC,280661.0,6
X-SC,267094.0,4


In [25]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,891849.0,6
M-SC,XY24,732334.0,6
H-SC,AR,537100.0,3
X-LC,X40,467569.0,4
M-SC,XR,426203.0,5
M-MC,XY24,380268.0,3
H-SC,XR,205957.0,3
H-MC,XY24,172137.0,2
M-SC,OX40N,164981.0,5


In [26]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
